In [2]:
import pandas as pd

# Reading data using pandas
df = pd.read_csv('Reliance.csv')



'''
Series - 1D Labeled array
DataFrames - 2D Labeled array

A DataFrame is a dictionary of Series under the hood

Both "Series" and "DataFrame" are subclass fo the NDFrame class

NDFrame is the base class in pandas for all data strucutres that handle labeled, array like data

'''
df[['Date']]

type(df[['Open','High']])

df[['Open','High']]


,Open,High
0,1469.1,1498.8
1,1464.0,1472.4
2,1465.0,1475.0
3,1453.0,1463.8
4,1431.0,1471.0
